In [ ]:
import os, pandas as pd, re

In [ ]:
INPUT_FOLDER = "json"

In [ ]:
all_pdfs = []
for root, folder, files in os.walk(INPUT_FOLDER):
    for file_name in files:
        if not file_name.endswith("pdf.json"):
            continue
        all_pdfs.append(re.sub(r'.json$', '', file_name))

In [ ]:
df_annotated = pd.DataFrame({"file":all_pdfs})

In [ ]:
re_nct = re.compile(r'NCT\d+')
def get_nct(filename):
    return re_nct.findall(filename)[0]
df_annotated["nct_id"] = df_annotated["file"].apply(get_nct)

In [ ]:
def assign_to_dataset(nct_id):
    if nct_id[5] == "8":
        return "val"
    if nct_id[5] == "9":
        return "test"
    return "train"
df_annotated["train_val"] = df_annotated.nct_id.apply(assign_to_dataset)

In [ ]:
import numpy as np
for item in "num_arms", "num_subjects", "phase", "intervention_type", "country":
    df = pd.read_csv("annotations/" + item + ".csv")
    
    df_annotated[item] = df_annotated.nct_id.map(df.set_index("nct_id")[item])

In [ ]:
df_annotated = df_annotated[~pd.isna(df_annotated.intervention_type)]

In [ ]:
df_annotated["has_sap"] = df_annotated.file.str.contains("SAP").apply(int)

In [ ]:
df_annotated.sort_values("file", inplace=True)

In [ ]:
df_annotated.drop_duplicates(subset="nct_id", keep='last', inplace=True)

In [ ]:
df_annotated.to_csv("annotations/all_annotations.csv", index=False)

In [ ]:
df_annotated[df_annotated.train_val.isin({"train", "val"})].drop(columns=["intervention_type","country"]).to_csv("for_freelancer/annotations_train_val.csv", index=False)